# Geospatial Analysis 

In this section, we will cover some super basic geospatial analysis with Python. Since we have latitude and longitude coordinates with some of our bird strike data, it might make sense if we plot it to see if we can spot some geographic trends. Granted, doing thorough analysis would warrant bringing in flight schedules and other outside data, but we will learn just enough to get our feet wet. 

## Basic Map Using Geopandas 

**Geospatial analysis** is using GIS and map layers to make sense of geographic patterns in data. It's working with geographic maps. There is a useful library called Geopandas for dealing with geospatial analysis, and it extends Pandas with a simple and clever feature: it adds a `geometry` column to given dataframe. 

Getting mapping data can be pretty involved, but there is a [Natural Earth](https://www.naturalearthdata.com/) site that provides some free mapping datasets to work with. I have downloaded a few of these datasets already alongside this notebook. Let's bring in the "countries" file and display it. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Download the world shapefile from Natural Earth's website
world = gpd.read_file(r"resource/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# Plot the world map
ax = world.plot()

# Show the plot
plt.show()

That gives us a map of the world. Very nice. But what exactly is this object we read from a file in `geopandas`? 

In [ ]:
world

If this looks like a `DataFrame`, that's because it is. More correctly, it is a `GeoDataFrame` and what sets it apart is it has a `geometry` column to handle the mapping representation of each record. 

There are some familiar and useful Pandas functions we can do with a `GeoDataFrame`. For example, I can only grab records that are sovereign to the United States of America. 

In [ ]:
world[world["SOVEREIGNT"] == "United States of America"]

Interesting. We got back the United States itself as well as its territory Puerto Rico. We can also just plot those results. 

In [ ]:
usa = world[(world["SOVEREIGNT"] == "United States of America")]

# Plot the world map
ax = usa.plot()

Let's zoom back out and bring in not just the whole world, but also all the bird strike data we have latitude and longtiudinal data for. Recall that we do not have latitude and longitude values for every record. Let's get that count. 

In [ ]:
import pandas as pd 

df = pd.read_csv('birdstrike_section2.csv')

((df["LATITUDE"].notna()) & df["LONGITUDE"].notna()).value_counts()

We have 121,359 records with latitude and longitude values, and 19,710 records that do not. We can live with that and explore why those locations were not provided. But we will let our plots ignore those missing values. 

Let's now create a separate `GeoDataFrame` called `gdf` that will take that bird strike data, but then add the `geometry` column using the `LONGITUDE` and `LATITUDE` columns. GeoPandas has a convenient function `points_from_xy` that will do that conversion. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Download the world shapefile from Natural Earth's website
world = gpd.read_file(r"resource/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df["LONGITUDE"],y=df["LATITUDE"]),crs="EPSG:4326")

# Plot the world map
ax = world.plot()

gdf.plot(ax=ax, color='red', markersize=5)

# Show the plot
plt.show()

And just for curiosity's sake, let's take a look at that `GeoDataFrame`. Sure enough, it has that `geometry` column which allows the plot above to be possible. 

In [ ]:
with pd.option_context('display.max_columns', None):
    display(gdf)

We can "zoom in" on just the United States if we like, as after all this data comes from an American agency: the FAA. This means we are going to see a heavy bias of US incidents since the FAA only tracks flights that are coming or going from the US. We can use a spatial join `sjoin()` to only include bird strikes that exist within the shapes of the States. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd 

usa = world[(world["SOVEREIGNT"] == "United States of America") & (world["TYPE"] == "Country")]

# Plot the world map
ax = usa.plot()

# Plot only the USA points
usa_points = gpd.sjoin(gdf, usa, predicate='within')
usa_points.plot(ax=ax, color='red', markersize=.1)

# Show the plot
plt.show()

## Provinces and States

Let's say we wanted to zoom in further. Let's bring in the states and provinces dataset. 

In [ ]:
states = gpd.read_file(r"resource/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")
states

Notice when we plot the states, we now see borders. 

In [ ]:
states.plot()

Let's remove Alaska and Hawaii, and do a spatial join with only bird strikes in the mainland. 

In [ ]:
mainland = states[~states['name'].isin(['Alaska', 'Hawaii'])]

mainland_points = gpd.sjoin(gdf, mainland, predicate='within')

ax = mainland.plot()
mainland_points.plot(ax=ax, color='red', markersize=.1)

plt.show()

For this next part, I want to also see the airports and see if bird strikes (unsurprisingly) happen near airports. Recall bird strikes are more likely to happen during takeoff and approach phases. Let's bring in this airport data I got from the FAA. 

In [ ]:
airports = pd.read_csv('resource/Airports.csv', usecols=['Icao_Ident','the_geom'])
airports 

I only pulled in two columns, and notice that one of the columns has a column called `the_geom` which we would think would work with GeoPandas. For some reason it wasn't. So I am going to do some cleanup separating out those `Longitude` and `Latitude` columns with regular expressions, removing the boilerplate like the parantheses and `Point`, and then build a new GeoDataFrame around those two values. 

In [ ]:
airports[['Longitude', 'Latitude']] = airports['the_geom'].str.split(r"(?<=[0-9])[ ](?=[-0-9])",regex=True, expand=True)

airports['Longitude'] = airports['Longitude'].str.replace('[^-0-9.]','', regex=True)
airports['Latitude'] = airports['Latitude'].str.replace('[^-0-9.]','', regex=True)

airports = gpd.GeoDataFrame(airports, geometry=gpd.points_from_xy(x=airports["Longitude"],y=airports["Latitude"]),crs="EPSG:4326")
airports

Now I can take a look at bird strikes alongside the airports. I am going to make the airports white dots, and for focus I am going to look at the state of Texas. 

In [ ]:
texas = states[states['name'].isin(['Texas'])]

texas_points = gpd.sjoin(gdf, texas, predicate='within')
texas_airports = gpd.sjoin(airports, texas, predicate='within')

ax = texas.plot()
texas_airports.plot(ax=ax, color='white', markersize=1)
texas_points.plot(ax=ax, color='red', markersize=2)

plt.show()

It does look like a lot of bird strikes where there are a cluster of more airports. I do wonder if that North Texas cluster has many bird strikes for DFW-related flights. DFW is famously busy on a global level. Can you go investigate that? 

Let's look at Florida too. 

In [ ]:
florida = states[states['name'].isin(['Florida'])]

florida_points = gpd.sjoin(gdf, florida, predicate='within')
florida_airports = gpd.sjoin(airports, florida, predicate='within')

ax = florida.plot()
florida_airports.plot(ax=ax, color='white', markersize=1)
florida_points.plot(ax=ax, color='red', markersize=2)

plt.show()

There's less of an obvious signal at Florida. It might help if we filter out airports that are small or tiny airstrips, and we could filter this by the number of flights per day. There's a lot of intersting analysis we could do here especially as we bring in more outside data! But in the interest of time we will wrap up here. I encourage you to chase down this data and keep exploring! 

## Exercise

Complete the code (by replacing the question marks "?") below to plot bird strikes and airports in the state of New York. 

In [ ]:
import geopandas as gpd
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv('birdstrike_section2.csv')
gdf = gpd.GeoDataFrame(?, geometry=gpd.points_from_xy(x=df["LONGITUDE"],y=df["LATITUDE"]),crs="EPSG:4326")
states = gpd.read_file(r"resource/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")

airports = pd.read_csv('resource/Airports.csv', usecols=['Icao_Ident','the_geom'])
airports[['Longitude', 'Latitude']] = airports['the_geom'].str.split(r"(?<=[0-9])[ ](?=[-0-9])",regex=True, expand=True)
airports['Longitude'] = airports['Longitude'].str.replace('[^-0-9.]','', regex=True)
airports['Latitude'] = airports['Latitude'].str.replace('[^-0-9.]','', regex=True)
airports = gpd.GeoDataFrame(?, geometry=gpd.points_from_xy(x=airports["Longitude"],y=airports["Latitude"]),crs="EPSG:4326")

new_york = states[states['name'].isin(['New York'])]

new_york_points = gpd.sjoin(?, ?, predicate=?)
new_york_airports = gpd.sjoin(?, ?, predicate=?)

ax = new_york.plot()
new_york_airports.plot(ax=ax, color='white', markersize=1)
new_york_points.plot(ax=ax, color='red', markersize=2)

plt.show()

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import geopandas as gpd
import pandas as pd 
import matplotlib.pyplot as plt

df = pd.read_csv('birdstrike_section2.csv')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df["LONGITUDE"],y=df["LATITUDE"]),crs="EPSG:4326")
states = gpd.read_file(r"resource/ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")

airports = pd.read_csv('resource/Airports.csv', usecols=['Icao_Ident','the_geom'])
airports[['Longitude', 'Latitude']] = airports['the_geom'].str.split(r"(?<=[0-9])[ ](?=[-0-9])",regex=True, expand=True)
airports['Longitude'] = airports['Longitude'].str.replace('[^-0-9.]','', regex=True)
airports['Latitude'] = airports['Latitude'].str.replace('[^-0-9.]','', regex=True)
airports = gpd.GeoDataFrame(airports, geometry=gpd.points_from_xy(x=airports["Longitude"],y=airports["Latitude"]),crs="EPSG:4326")

new_york = states[states['name'].isin(['New York'])]

new_york_points = gpd.sjoin(gdf, new_york, predicate='within')
new_york_airports = gpd.sjoin(airports, new_york, predicate='within')

ax = new_york.plot()
new_york_airports.plot(ax=ax, color='white', markersize=1)
new_york_points.plot(ax=ax, color='red', markersize=2)

plt.show()